In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# pip install xgboost --no-binary xgboost -v

In [4]:
train = pd.read_csv('/kaggle/input/may5aaa/55data_train.csv')
tests = pd.read_csv('/kaggle/input/may5aaa/55data_tests.csv')

In [ ]:
train = pd.read_csv('/kaggle/input/raw-water/raw_train.csv')
tests = pd.read_csv('/kaggle/input/raw-water/raw_tests.csv')

In [ ]:
train_labels=train.pop('A厂')
train_prepared=train

In [5]:
from mlxtend.regressor import StackingRegressor

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [7]:
import time #implementing in this function the time spent on training the model
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

def modelfit(alg, dtrain, target, only_predict = False):
    #Fit the algorithm on the data
    time_start = time.perf_counter() #start counting the time
    if not only_predict:
        alg.fit(dtrain, target)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    
    kfolds = KFold(n_splits=6, shuffle=True, random_state=42)
    
    cv_score = cross_val_score(alg, dtrain,target, cv=kfolds, scoring='neg_mean_squared_error')
    cv_score = np.sqrt(-cv_score)
    
    time_end = time.perf_counter()
    
    total_time = time_end-time_start
    #Print model report:
    print("\nModel Report")
    print("RMSE :  {:.4f}".format(np.sqrt(mean_squared_error(target, dtrain_predictions))))
    print("CV Score : Mean -  %.4f | Std -  %.4f | Min -  %.4f | Max - %.4f" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    print("Amount of time spent during training the model and cross validation: %4.3f seconds" % (total_time))
    
def plot_feature_importance(model, df):
    feature_importance = model.feature_importances_[:30]
    # make importances relative to max importance
    plt.figure(figsize=(20, 20)) #figure size
    feature_importance = 100.0 * (feature_importance / feature_importance.max()) #making it a percentage relative to the max value
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, df.columns[sorted_idx], fontsize=15) #used train_drop here to show the name of each feature instead of our train_prepared 
    plt.xlabel('Relative Importance', fontsize=20)
    plt.ylabel('Features', fontsize=20)
    plt.title('Variable Importance', fontsize=30)

In [8]:
lin_reg = LinearRegression()
modelfit(lin_reg, train_prepared, train_labels)

NameError: name 'train_prepared' is not defined

In [9]:
forest_reg = RandomForestRegressor(n_estimators=300, 
                                   random_state=1026, 
                                   min_samples_leaf=5,
                                   min_samples_split = 5,
                                   max_depth = 15,
                                   n_jobs=-1, oob_score=True)
modelfit(forest_reg, train_prepared, train_labels)

NameError: name 'train_prepared' is not defined

In [ ]:
plot_feature_importance(forest_reg, train_prepared)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=421, max_depth = 15, min_samples_split = 5)
modelfit(tree_reg, train_prepared, train_labels) 

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

# tree_ada = DecisionTreeRegressor(random_state = 42,max_depth = 4)

ada_reg = AdaBoostRegressor(
    tree_reg, n_estimators=300, random_state=42,learning_rate=0.009, loss='square')
modelfit(ada_reg, train_prepared, train_labels)

# 181.589

In [ ]:
plot_feature_importance(ada_reg, train_prepared)

In [ ]:
params = {'n_estimators': 100, 'max_depth': 14, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params_1 = {'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth' : 14,
            'max_features': 'sqrt', 'min_samples_leaf': 15, 'min_samples_split': 5, 
            'loss': 'huber', 'random_state': 42}

# train_dummies_prepared = num_pipeline.fit_transform(train_dummies)

gdb_model = GradientBoostingRegressor(**params_1)
modelfit(gdb_model, train_prepared, train_labels)

---

# now 

In [5]:
train = pd.read_csv('/kaggle/input/may5aaa/55data_train.csv')
tests = pd.read_csv('/kaggle/input/may5aaa/55data_tests.csv')

In [6]:
train_raw2 = train
tests_raw2 = tests

In [7]:
train_raw2['B厂'] = train_raw2['B厂'].astype(int) 
tests_raw2['B厂'] = tests_raw2['B厂'].astype(int) 
# for i in ['B厂', 'year', 'wee']:
#     train_raw2[i] = train_raw2[i].astype(object)
#     tests_raw2[i] = tests_raw2[i].astype(object)

tests_raw2.columns = train_raw2.columns

In [8]:
train_raw2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070 entries, 0 to 2069
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A厂      2070 non-null   float64
 1   B厂      2070 non-null   int64  
 2   year    2070 non-null   int64  
 3   month   2070 non-null   int64  
 4   date    2070 non-null   int64  
 5   wee     2070 non-null   int64  
 6   sum     2070 non-null   float64
 7   sub     2070 non-null   float64
 8   div     2070 non-null   float64
 9   log     2070 non-null   float64
dtypes: float64(5), int64(5)
memory usage: 161.8 KB


In [10]:
y = train_raw2.pop('A厂')
X = train_raw2
tests_raw2.pop('A厂')

0      1.926031
1      1.926081
2      1.926131
3      1.926180
4      1.926230
         ...   
297   -0.299085
298   -0.299056
299   -0.299028
300   -0.298999
301   -0.298970
Name: A厂, Length: 302, dtype: float64

In [64]:
y = np.array(y).reshape(-1,1)

In [12]:
tests_raw2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   B厂      302 non-null    int64  
 1   year    302 non-null    int64  
 2   month   302 non-null    int64  
 3   date    302 non-null    int64  
 4   wee     302 non-null    int64  
 5   sum     302 non-null    float64
 6   sub     302 non-null    float64
 7   div     302 non-null    float64
 8   log     302 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 21.4 KB


In [67]:
meta_random_seed = 1026529

In [40]:
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_percentage_error

In [80]:
def objective(trial, X=X, y=y):

    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=meta_random_seed)


    lgb_params={
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2),
        'max_depth': trial.suggest_int('max_depth', 6, 31),
        'num_leaves': trial.suggest_int('num_leaves', 31, 120),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-1, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-1, 1),
        'random_state': meta_random_seed,
        'metric': 'mse',
        'n_estimators': trial.suggest_int('n_estimators', 6, 3000),
        'n_jobs': -1,
        'cat_feature': [x for x in range(len(categorical_columns))],
        'bagging_seed': 2021,
        'feature_fraction_seed': 2021,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 500),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'max_bin': trial.suggest_int('max_bin', 128, 1024),
        'min_data_per_group': trial.suggest_int('min_data_per_group', 5, 50),
        'cat_smooth': trial.suggest_int('cat_smooth', 10, 250),
        'cat_l2': trial.suggest_int('cat_l2', 1, 20)
    }

    lgb = LGBMRegressor(
        **lgb_params
    )
    lgb.fit(
        X_train,
        y_train,
        eval_set=(X_test,y_test),
        eval_metric='mean_absolute_percentage_error',
        early_stopping_rounds=100,
        verbose=False
    )
    predictions=lgb.predict(X_test)

    return mean_absolute_percentage_error(y_test, predictions)

In [81]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, timeout=3600*7, n_trials=25)

[I 2021-05-05 06:57:42,005] A new study created in memory with name: no-name-75e64e44-ecaa-4fd5-8827-9d5dc3a2cab8
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
[I 2021-05-05 06:57:43,396] Trial 0 finished with value: 0.8466679394775066 and parameters: {'learning_rate': 0.001742788591210247, 'max_depth': 143, 'num_leaves': 78, 'reg_alpha': 0.9133738623181126, 'reg_lambda': 0.8756985578508477, 'n_estimators': 2837, 'colsample_bytree': 0.7033306741101055, 'min_child_samples': 106, 'subsample_freq': 3, 'subsample': 0.7421045833839981, 'max_bin': 807, 'min_data_per_group': 34, 'cat_smooth': 89, 'cat_l2': 13}. Best is trial 0 with value: 0.8466679394775066.
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
[I 2021-05-05 06:57:44,990] Trial 1 finished with va

In [82]:
study.best_params

# {'learning_rate': 0.0026955685638868463,
#  'max_depth': 180,
#  'num_leaves': 110,
#  'reg_alpha': 0.8365820878758264,
#  'reg_lambda': 0.5052568831768142,
#  'n_estimators': 2458,
#  'colsample_bytree': 0.7589370718114493,
#  'min_child_samples': 7,
#  'subsample_freq': 10,
#  'subsample': 0.7036373549101399,
#  'max_bin': 906,
#  'min_data_per_group': 50,
#  'cat_smooth': 242,
#  'cat_l2': 4}

{'learning_rate': 0.0026955685638868463,
 'max_depth': 180,
 'num_leaves': 110,
 'reg_alpha': 0.8365820878758264,
 'reg_lambda': 0.5052568831768142,
 'n_estimators': 2458,
 'colsample_bytree': 0.7589370718114493,
 'min_child_samples': 7,
 'subsample_freq': 10,
 'subsample': 0.7036373549101399,
 'max_bin': 906,
 'min_data_per_group': 50,
 'cat_smooth': 242,
 'cat_l2': 4}

In [13]:
lgbm_params = {'learning_rate': 0.0026955685638868463,
 'max_depth': 180,
 'num_leaves': 110,
 'reg_alpha': 0.8365820878758264,
 'reg_lambda': 0.5052568831768142,
 'n_estimators': 2458,
 'colsample_bytree': 0.7589370718114493,
 'min_child_samples': 7,
 'subsample_freq': 10,
 'subsample': 0.7036373549101399,
 'max_bin': 906,
 'min_data_per_group': 50,
 'cat_smooth': 242,
 'cat_l2': 4}

In [124]:
def objective2(trial, X=X, y=y):
    X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25,random_state=meta_random_seed)
    
    xgb_params={
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2),
        'max_depth': trial.suggest_int('max_depth', 6, 31),
        'alpha': trial.suggest_float('alpha', 1e-1, 1),
        'gamma': trial.suggest_float('gamma', 1e-1, 1),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-1, 1),
        'random_state': meta_random_seed,
        'n_estimators': trial.suggest_int('n_estimators', 6, 3000),
        'n_jobs': -1,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 500),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'max_bin': trial.suggest_int('max_bin', 128, 1024)       
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 350, 1000),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.10),
#         'subsample': trial.suggest_uniform('subsample', 0.50, 0.90),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 0.90),
#         'gamma': trial.suggest_int('gamma', 0, 20),
#         'missing': -999,
#         'tree_method': 'gpu_hist'  
#     }
    }

    xgb = XGBRegressor(
        **xgb_params,tree_method = 'gpu_hist'
    )
    xgb.fit(
        X_train,
        y_train,
        eval_set=[(X_test,y_test)],
        eval_metric='rmse',
        early_stopping_rounds=100,
        verbose=False
    )
    predictions=xgb.predict(X_test)

    return mean_absolute_percentage_error(y_test, predictions)

In [125]:
study2 = optuna.create_study(direction='minimize')
study2.optimize(objective2, timeout=3600*6, n_trials=15)

[I 2021-05-05 07:24:56,695] A new study created in memory with name: no-name-b55a00ba-7d79-4dc4-8814-01c6a652f86c
/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
[I 2021-05-05 07:24:57,968] Trial 0 finished with value: 2.85772015370366 and parameters: {'learning_rate': 0.002222756793278268, 'max_depth': 11, 'alpha': 0.39013442018130384, 'gamma': 0.8049052819100202, 'n_estimators': 1060, 'colsample_bytree': 0.8087221109310607, 'min_child_weight': 386, 'subsample': 0.861113748957127, 'max_bin': 956}. Best is trial 0 with value: 2.85772015370366.
/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copi

XGBoostError: [07:31:52] ../src/tree/updater_gpu_hist.cu:793: Exception in gpu_hist: [07:31:52] ../src/tree/param.h:249: Check failed: n_nodes != 0 (0 vs. 0) : 
Stack trace:
  [bt] (0) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x912df) [0x7f997f92b2df]
  [bt] (1) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x276507) [0x7f997fb10507]
  [bt] (2) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4deae1) [0x7f997fd78ae1]
  [bt] (3) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4e8df4) [0x7f997fd82df4]
  [bt] (4) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x18c792) [0x7f997fa26792]
  [bt] (5) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x18f07d) [0x7f997fa2907d]
  [bt] (6) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x1b9ac3) [0x7f997fa53ac3]
  [bt] (7) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+

In [1]:
#study.best_params
study2.best_params

NameError: name 'study2' is not defined

In [14]:
xgb_params = {'learning_rate': 0.0018800314311429142,
 'max_depth': 16,
 'alpha': 0.2516299216083434,
 'gamma': 0.9382629503388609,
 'n_estimators': 1519,
 'colsample_bytree': 0.8616778784529846,
 'min_child_weight': 132,
 'subsample': 0.7089219366903059,
 'max_bin': 332}

In [19]:
#

In [17]:
train_raw2

,B厂,year,month,date,wee,sum,sub,div,log
0,0,1,1,2018,1,-0.678731,-0.939331,1.059117,1.004726
1,0,1,2,2018,1,-0.040439,-1.072916,1.034185,1.002742
2,0,1,3,2018,1,0.166579,-1.089144,1.030923,1.002478
3,0,1,4,2018,1,-0.047661,-1.187690,1.017847,1.001442
4,0,1,5,2018,1,-0.359147,-1.013835,0.957121,0.996423
...,...,...,...,...,...,...,...,...,...
2065,1,10,27,2020,1,1.475312,0.009076,1.163365,1.012207
2066,1,10,28,2020,1,1.361221,-0.130746,1.146982,1.011069
2067,1,10,29,2020,1,1.201056,-0.272881,1.130873,1.009936
2068,1,10,30,2020,1,1.478330,0.290262,1.201476,1.014828


In [29]:
train_df = X
test_df = tests_raw2

In [37]:
from sklearn.model_selection import StratifiedKFold,RepeatedKFold
from scipy.special import expit
from sklearn.calibration import CalibratedClassifierCV

random_state = 1026529
n_folds = 6
k_fold = RepeatedKFold(n_splits=n_folds, random_state=random_state)

# y = train_df["target"]

xgb_train_preds = np.zeros(len(train_df.index), )
xgb_test_preds = np.zeros(len(test_df.index), )
# xgb_features = xgb_cat_features + list(numerical_columns)

lgbm_train_preds = np.zeros(len(train_df.index), )
lgbm_test_preds = np.zeros(len(test_df.index), )
# lgbm_features = lgb_cat_features + list(numerical_columns)

# cb_train_preds = np.zeros(len(train_df.index), )
# cb_test_preds = np.zeros(len(test_df.index), )
# cb_features = cb_cat_features + list(numerical_columns)

# ridge_train_preds = np.zeros(len(train_df.index), )
# ridge_test_preds = np.zeros(len(test_df.index), )
# ridge_features = ridge_cat_features + list(numerical_columns)

In [42]:
for fold, (train_index, test_index) in enumerate(k_fold.split(train_df, y)):
    print("--> Fold {}".format(fold + 1))
    y_train = y.iloc[train_index]
    y_valid = y.iloc[test_index]

    ########## Generate train and valid sets ##########
    xgb_x_train = pd.DataFrame(train_df.iloc[train_index])
    xgb_x_valid = pd.DataFrame(train_df.iloc[test_index])

    lgbm_x_train = pd.DataFrame(train_df.iloc[train_index])
    lgbm_x_valid = pd.DataFrame(train_df.iloc[test_index])
    
#     cb_x_train = pd.DataFrame(train_df[cb_features].iloc[train_index])
#     cb_x_valid = pd.DataFrame(train_df[cb_features].iloc[test_index])

#     ridge_x_train = pd.DataFrame(train_df[ridge_features].iloc[train_index])
#     ridge_x_valid = pd.DataFrame(train_df[ridge_features].iloc[test_index])

    ########## XGBoost model ##########
    xgb_model = XGBRegressor(
        seed=random_state,
        n_estimators=1519,
#         verbosity=1,
        eval_metric="rmse",
        tree_method="gpu_hist",
        gpu_id=0,
        alpha=0.2516299216083434,
        colsample_bytree=0.8616778784529846,
        gamma=0.9382629503388609,
        reg_lambda=0.969826765347235612,
        learning_rate=0.0018800314311429142,
        max_bin=332,
        max_depth=16,
        min_child_weight=132,
        subsample=0.7089219366903059,
    )
    xgb_model.fit(
        xgb_x_train,
        y_train,
        eval_set=[(xgb_x_valid, y_valid)], 
        verbose=0,
        early_stopping_rounds=200
    )

    train_oof_preds = xgb_model.predict(xgb_x_valid)
    test_oof_preds = xgb_model.predict(test_df)
    xgb_train_preds[test_index] = train_oof_preds
    xgb_test_preds += test_oof_preds / n_folds
    
    print(": XGB - RMSE = {}".format(mean_absolute_percentage_error(y_valid, train_oof_preds)))
    
    ########## LGBM model ##########
    lgbm_model = LGBMRegressor(
        random_state=random_state,
        cat_l2=4,
        cat_smooth=242,
        colsample_bytree=0.7589370718114493,
        early_stopping_round=200,
        learning_rate=0.0026955685638868463,
        max_bin=906,
        max_depth=180,
        metric="rmse",
        min_child_samples=7,
        min_data_per_group=50,
        n_estimators=2458,
        n_jobs=-1,
        num_leaves=110,
        reg_alpha=0.8365820878758264,
        reg_lambda=0.5052568831768142,
        subsample=0.7036373549101399,
        subsample_freq=10,
        verbose=-1,
    )
    lgbm_model.fit(
        lgbm_x_train,
        y_train,
        eval_set=[(lgbm_x_valid, y_valid)], 
        verbose=0,
    )

    train_oof_preds = lgbm_model.predict(lgbm_x_valid)
    test_oof_preds = lgbm_model.predict(test_df)
    lgbm_train_preds[test_index] = train_oof_preds
    lgbm_test_preds += test_oof_preds / n_folds
    
    print(": LGB - RMSE = {}".format(mean_absolute_percentage_error(y_valid, train_oof_preds)))

--> Fold 1


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


: XGB - ROC AUC Score = 1.41972605943558
: LGB - ROC AUC Score = 0.28256243377861545
--> Fold 2


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


: XGB - ROC AUC Score = 0.730985982905113
: LGB - ROC AUC Score = 0.19142691932569186
--> Fold 3


...

: XGB - ROC AUC Score = 1.1303225217317392
: LGB - ROC AUC Score = 0.29200407564152026
--> Fold 59


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


: XGB - ROC AUC Score = 0.6753780857198867
: LGB - ROC AUC Score = 0.1684573305735969
--> Fold 60


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


: XGB - ROC AUC Score = 0.7330610409492407
: LGB - ROC AUC Score = 0.16540318308129962


In [55]:
pd.DataFrame(lgbm_test_preds/n_folds).to_csv('./lgbm55.csv')

In [56]:
pd.DataFrame(xgb_test_preds/n_folds).to_csv('./xgb55.csv')

---

# over 

In [ ]:
import xgboost as xgb

    
params = {'objective': 'reg:squarederror', 
              'eta': 0.01, 
              'max_depth': 16, 
              'subsample': 0.9, 
              'colsample_bytree': 1,  
              'eval_metric': 'rmse', 
              'seed': 42, 
              'silent': True,
    }

model_xgb = xgb.XGBRegressor(**params, n_estimators=1000,learning_rate=0.05,verbose_eval=True)

In [ ]:
def objective(trial,data=x,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25,random_state=1026s)
    
    # To select which parameters to optimize, please look at the XGBoost documentation:
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    param = {
        'device':'gpu',  # Use GPU acceleration
        'metric': 'rmse',
        'random_state': 42,
        'reg_lambda': trial.suggest_loguniform(
            'reg_lambda', 1e-3, 10.0
        ),
        'reg_alpha': trial.suggest_loguniform(
            'reg_alpha', 1e-3, 10.0
        ),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.3,0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical(
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300, 3000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,15,17,20]
        ),
        'min_child_samples': trial.suggest_int(
            'min_child_samples', 1, 300
        ),
        'num_leaves': trial.suggest_int(
            'num_leaves', 15, 120
        ),
    }
    model = LGBMRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)], early_stopping_rounds=300, verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
from datetime import datetime
fold_val_pred = []
fold_err = []
fold_importance = []
record = dict()
kfold = list(KFold(10,shuffle = True, random_state = 42).split(train_prepared))

def xgb_model(train, params, train_label, fold, verbose, pipeline):
    for i, (trn, val) in enumerate(fold) :
        print(i+1, "fold.    RMSE")
        trn_x = train.loc[trn, :]
        trn_y = train_label[trn]
        val_x = train.loc[val, :]
        val_y = train_label[val]
        fold_val_pred = []
    
        start = datetime.now()
        model = xgb.train(params
                      , xgb.DMatrix(trn_x, trn_y)
                      , 3500
                      , [(xgb.DMatrix(trn_x, trn_y), 'train'), (xgb.DMatrix(val_x, val_y), 'valid')]
                      , verbose_eval=verbose
                      , early_stopping_rounds=500
                      , callbacks = [xgb.callback.record_evaluation(record)])
        best_idx = np.argmin(np.array(record['valid']['rmse']))
        val_pred = model.predict(xgb.DMatrix(val_x), ntree_limit=model.best_ntree_limit)
        fold_val_pred.append(val_pred*0.2)
        fold_err.append(record['valid_0']['rmse'][best_idx])
        fold_importance.append(model.feature_importance('gain'))
        
        print("xgb model.", "{0:.5f}".format(result['error']), '(' + str(int((datetime.now()-start).seconds/60)) + 'm)')

In [ ]:
modelfit(model_xgb, train_prepared, train_labels)

#132.587 seconds

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
#     ('pca', PCA(5)),
    ('xgb', xgb.XGBRegressor())
])

param_grid = {
#     'pca__n_components': [3, 5, 7, 10,15],
    'xgb__n_estimators': [500,1000,2000,3500],
    'xgb__learning_rate': [0.09,0.1,0.05,0.001]
}

pipe.fit(train_prepared, train_labels)

In [ ]:
modelfit(pipe, train_prepared, train_labels, True)

In [ ]:
import lightgbm as lgb
import xgboost as xgb

lgb_model = lgb.LGBMRegressor(objective='regression', 
                                       num_leaves=500,
                                       learning_rate=0.01, 
                                       n_estimators=3000,
                                       max_depth = 15,
#                                        max_bin=200, 
                                       bagging_fraction=0.85,
                                       bagging_freq=5, 
                                       bagging_seed=75,
#                                        feature_fraction=1,
#                                        feature_fraction_seed=75,
                                       metric='rmse',
                                       verbose=-1,
                                       )

modelfit(lgb_model, train_prepared, train_labels)

In [ ]:
from mlxtend.regressor import StackingCVRegressor
models_total = (lin_reg, forest_reg, ada_reg, gdb_model, lgb_model)
# model_ada_gbd = (ada_reg,gdb_model)
stack_gen = StackingCVRegressor(regressors= models_total,
                                meta_regressor=model_xgb,
                                use_features_in_secondary=True)

In [ ]:
modelfit(stack_gen, train_prepared, train_labels)

In [ ]:
tests_labels=tests.pop('A厂')
tests_prepared=tests

In [ ]:
prediction_some_regressors = stack_gen.predict(tests_prepared)

In [ ]:
prediction_some_regressors

In [ ]:
pd.DataFrame(prediction_some_regressors).to_csv("./kag.csv", index = False)